In [1]:
# Normalise features
# Find the optimal parameters C (inverse regulalisation strengths)
# Find the best features to predict stock movement (up or down)
# Convert probability into absolute values (stock % increase/decrease)
# SVC, RandomForest, MinMax, LongShortTermMemory

%matplotlib inline
%pylab inline

# Nice Formatting within Jupyter Notebook
%matplotlib inline
from IPython.display import display # Allows multiple displays from a single code-cell
from jupyterthemes import jtplot

#import classes
from utilities import *
from company import Company
from onestep_baseline_company import OneStepBaselineCompany
from onestep_lstm_company import OneStepLSTMCompany
from multistep_baseline_company import MultiStepBaselineCompany
from multistep_lstm_company import MultiStepLSTMCompany

jtplot.style(theme='grade3')
jtplot.style(theme='grade3')
jtplot.style(theme='grade3')


Populating the interactive namespace from numpy and matplotlib


Using TensorFlow backend.


In [2]:
import tensorflow as tf
import os
from keras import backend as K
config = tf.ConfigProto(intra_op_parallelism_threads=NUM_PARALLEL_EXEC_UNITS, inter_op_parallelism_threads=2,
                        allow_soft_placement=True, device_count={'CPU': NUM_PARALLEL_EXEC_UNITS})

session = tf.Session(config=config)
K.set_session(session)
os.environ["OMP_NUM_THREADS"] = "NUM_PARALLEL_EXEC_UNITS"
os.environ["KMP_BLOCKTIME"] = "30"
os.environ["KMP_SETTINGS"] = "1"
os.environ["KMP_AFFINITY"] = "granularity=fine,verbose,compact,1,0"
    
start_train_date = "01/01/2010"
end_train_start_test_date = "01/01/2015"
end_test_date = "01/01/2020"

obj = MultiStepLSTMCompany("AAPL", start_train_date, end_train_start_test_date, end_test_date, 
                         n_lag=3, n_seq=3, n_epochs=3000, n_batch=17, n_neurons=4, tech_indicators="all", model_type="vanilla")
obj.train()

Preprocessing the data
Retrieved price series and raw pd from disk
Add from existing raw_pd  ad
Add from existing raw_pd  adosc
Add from existing raw_pd  adx
Add from existing raw_pd  adxr
Add from existing raw_pd  apo
Add from existing raw_pd  aroon
Add from existing raw_pd  aroonosc
Add from existing raw_pd  bbands
Add from existing raw_pd  bop
Add from existing raw_pd  cci
Add from existing raw_pd  cmo
Add from existing raw_pd  dema
Add from existing raw_pd  dx
Add from existing raw_pd  ema
Add from existing raw_pd  ht_dcperiod
Add from existing raw_pd  ht_dcphase
Add from existing raw_pd  ht_phasor
Add from existing raw_pd  ht_sine
Add from existing raw_pd  ht_trendline
Add from existing raw_pd  ht_trendmode
Add from existing raw_pd  kama
Add from existing raw_pd  macd
Add from existing raw_pd  macdext
Add from existing raw_pd  mama
Add from existing raw_pd  mfi
Add from existing raw_pd  midpoint
Add from existing raw_pd  midprice
Add from existing raw_pd  minus_di
Add from existin

In [3]:
#start_train_date = "01/01/2010"
#end_train_start_test_date = "01/01/2015"
#end_test_date = "01/01/2020"
#obj.update_train_test_set(start_train_date, end_train_start_test_date, end_test_date)
predictions = obj.predict()

trend_score = obj.score(metric="trend", predictions=predictions)

lstm_score = obj.score(metric="rmse", predictions=predictions)
apre_score = obj.score(metric="apre", predictions=predictions)
print("LSTM trend Score: ", trend_score)
print("LSTM RMSE Score: US dollar", lstm_score)
print("LSTM APRE Score: US dollar", apre_score)

actual [[101.5264  98.6662  98.6755]
 [ 98.6662  98.6755 100.0591]
 [ 98.6755 100.0591 103.9036]
 ...
 [171.06   172.97   174.23  ]
 [172.97   174.23   174.33  ]
 [174.23   174.33   174.87  ]]
predicted [[101.96029206 101.4426764  101.08357754]
 [101.59408444 101.67171418 101.77068044]
 [ 98.33198717  95.80521253  97.59663638]
 ...
 [172.28823687 172.55217887 172.79451978]
 [170.84825142 170.65266624 170.54138265]
 [173.15666649 173.35063086 173.53912112]]

Calculating trend score for  1
Correct counts:  522   Size of test set: 1045

Calculating trend score for  2
Correct counts:  515   Size of test set: 1045

Calculating trend score for  3
Correct counts:  511   Size of test set: 1045
LSTM trend Score:  [0.4995215311004785, 0.49282296650717705, 0.48899521531100476]
actual [[101.5264  98.6662  98.6755]
 [ 98.6662  98.6755 100.0591]
 [ 98.6755 100.0591 103.9036]
 ...
 [171.06   172.97   174.23  ]
 [172.97   174.23   174.33  ]
 [174.23   174.33   174.87  ]]
predicted [[101.96029206 101.4

In [6]:
start_train_date = "01/01/2010"
end_train_start_test_date = "01/01/2015"
end_test_date = "01/01/2020"

obj2 = MultiStepLSTMCompany("AAPL", start_train_date, end_train_start_test_date, end_test_date, 
                         n_lag=3, n_seq=3, n_epochs=3000, n_batch=17, n_neurons=4, tech_indicators="all", model_type="stacked")
obj2.train()

Preprocessing the data
Retrieved price series and raw pd from disk
Add from existing raw_pd  ad
Add from existing raw_pd  adosc
Add from existing raw_pd  adx
Add from existing raw_pd  adxr
Add from existing raw_pd  apo
Add from existing raw_pd  aroon
Add from existing raw_pd  aroonosc
Add from existing raw_pd  bbands
Add from existing raw_pd  bop
Add from existing raw_pd  cci
Add from existing raw_pd  cmo
Add from existing raw_pd  dema
Add from existing raw_pd  dx
Add from existing raw_pd  ema
Add from existing raw_pd  ht_dcperiod
Add from existing raw_pd  ht_dcphase
Add from existing raw_pd  ht_phasor
Add from existing raw_pd  ht_sine
Add from existing raw_pd  ht_trendline
Add from existing raw_pd  ht_trendmode
Add from existing raw_pd  kama
Add from existing raw_pd  macd
Add from existing raw_pd  macdext
Add from existing raw_pd  mama
Add from existing raw_pd  mfi
Add from existing raw_pd  midpoint
Add from existing raw_pd  midprice
Add from existing raw_pd  minus_di
Add from existin

In [8]:

predictions = obj2.predict()
apre_score = obj2.score(metric="apre", predictions=predictions)
trend_score = obj2.score(metric="trend", predictions=predictions)
rmsq_score = obj2.score(metric="rmse", predictions=predictions)
print("LSTM trend Score: ", trend_score)
print("LSTM RMSE Score: US dollar", rmsq_score)
print("LSTM APRE Score: US dollar", apre_score)


actual [[101.5264  98.6662  98.6755]
 [ 98.6662  98.6755 100.0591]
 [ 98.6755 100.0591 103.9036]
 ...
 [171.06   172.97   174.23  ]
 [172.97   174.23   174.33  ]
 [174.23   174.33   174.87  ]]
predicted [[102.28722586 102.28047457 102.05437247]
 [101.88475633 101.51001891 101.69486812]
 [ 99.00930956  99.10011615  99.14174236]
 ...
 [172.10843582 172.11350038 172.20384407]
 [170.58685485 170.82933104 171.05860578]
 [173.06455502 173.21387487 173.11337216]]
t+1 APRE: 0.012056
t+2 APRE: 0.016928
t+3 APRE: 0.021238
actual [[101.5264  98.6662  98.6755]
 [ 98.6662  98.6755 100.0591]
 [ 98.6755 100.0591 103.9036]
 ...
 [171.06   172.97   174.23  ]
 [172.97   174.23   174.33  ]
 [174.23   174.33   174.87  ]]
predicted [[102.28722586 102.28047457 102.05437247]
 [101.88475633 101.51001891 101.69486812]
 [ 99.00930956  99.10011615  99.14174236]
 ...
 [172.10843582 172.11350038 172.20384407]
 [170.58685485 170.82933104 171.05860578]
 [173.06455502 173.21387487 173.11337216]]

Calculating trend sc

In [9]:
start_train_date = "01/01/2010"
end_train_start_test_date = "01/01/2015"
end_test_date = "01/01/2020"

obj3 = MultiStepLSTMCompany("AAPL", start_train_date, end_train_start_test_date, end_test_date, 
                         n_lag=3, n_seq=3, n_epochs=3000, n_batch=17, n_neurons=4, tech_indicators="all", model_type="bi")
obj3.train()

Preprocessing the data
Retrieved price series and raw pd from disk
Add from existing raw_pd  ad
Add from existing raw_pd  adosc
Add from existing raw_pd  adx
Add from existing raw_pd  adxr
Add from existing raw_pd  apo
Add from existing raw_pd  aroon
Add from existing raw_pd  aroonosc
Add from existing raw_pd  bbands
Add from existing raw_pd  bop
Add from existing raw_pd  cci
Add from existing raw_pd  cmo
Add from existing raw_pd  dema
Add from existing raw_pd  dx
Add from existing raw_pd  ema
Add from existing raw_pd  ht_dcperiod
Add from existing raw_pd  ht_dcphase
Add from existing raw_pd  ht_phasor
Add from existing raw_pd  ht_sine
Add from existing raw_pd  ht_trendline
Add from existing raw_pd  ht_trendmode
Add from existing raw_pd  kama
Add from existing raw_pd  macd
Add from existing raw_pd  macdext
Add from existing raw_pd  mama
Add from existing raw_pd  mfi
Add from existing raw_pd  midpoint
Add from existing raw_pd  midprice
Add from existing raw_pd  minus_di
Add from existin

In [10]:
predictions = obj3.predict()
apre_score = obj3.score(metric="apre", predictions=predictions)
trend_score = obj3.score(metric="trend", predictions=predictions)
rmsq_score = obj3.score(metric="rmse", predictions=predictions)
print("LSTM trend Score: ", trend_score)
print("LSTM RMSE Score: US dollar", rmsq_score)
print("LSTM APRE Score: US dollar", apre_score)

actual [[101.5264  98.6662  98.6755]
 [ 98.6662  98.6755 100.0591]
 [ 98.6755 100.0591 103.9036]
 ...
 [171.06   172.97   174.23  ]
 [172.97   174.23   174.33  ]
 [174.23   174.33   174.87  ]]
predicted [[104.50104276 104.8879323  103.39409529]
 [101.26452534 102.61300346 101.2029194 ]
 [ 99.78030002 102.05271089  99.65490558]
 ...
 [170.72559286 169.46615259 169.11181018]
 [168.50901935 171.12979418 168.15822457]
 [172.89597826 172.11689991 170.58656718]]
t+1 APRE: 0.016578
t+2 APRE: 0.022247
t+3 APRE: 0.025615
actual [[101.5264  98.6662  98.6755]
 [ 98.6662  98.6755 100.0591]
 [ 98.6755 100.0591 103.9036]
 ...
 [171.06   172.97   174.23  ]
 [172.97   174.23   174.33  ]
 [174.23   174.33   174.87  ]]
predicted [[104.50104276 104.8879323  103.39409529]
 [101.26452534 102.61300346 101.2029194 ]
 [ 99.78030002 102.05271089  99.65490558]
 ...
 [170.72559286 169.46615259 169.11181018]
 [168.50901935 171.12979418 168.15822457]
 [172.89597826 172.11689991 170.58656718]]

Calculating trend sc

In [13]:
obj3.supervised_pd.shape[2]

IndexError: tuple index out of range